# TRT KPI: Minimize Time to Science
### *Number of lines of code for a given workflow in the cloud / Number of lines for same workflow on-prem
### *Run time for a given workflow in the cloud / run time for same workflow on-prem

#### Initial attempt: 
* Basic data search, customize, access, plotting, with a collection that is available on-prem and in Earthdata Cloud.
* Initially, single ICESat-2 data product: ATL03 v003
  * C1234714691-EEDTEST -> Harmony-enabled, UAT
  * C1233596099-NSIDC_TS1 -> EGI-enabled, UAT
* Bonus: Coincident data search/access/plot with another product
* Bonus: Different data access options beyond Harmony (direct s3, cloudfront, OPeNDAP)

## Import packages

In [24]:
import sys
import requests
from getpass import getpass
from platform import system
from netrc import netrc
from os.path import join, expanduser
from urllib import request
from http.cookiejar import CookieJar
import json
from pprint import pprint
import os
import time
import zipfile
import io
import shutil
import h5py
from pathlib import Path
import pandas as pd
import numpy as np
import cartopy
from matplotlib import pyplot as plt

## __On Prem workflow__

### (1. Search by time and bounding box (attempt using cmr-python library))

In [ ]:
%cd eo-metadata-tools/CMR/python
#!{sys.executable} -m pip3 install wheel 
#!{sys.executable} -m pip install wheel
#!{sys.executable} -m pip install https://github.com/nasa/eo-metadata-tools/releases/download/latest-master/eo_metadata_tools_cmr-0.0.1-py3-none-any.whl
%%bash
./runme.sh -p -i

import cmr.search.granule as gran

In [ ]:
#gran.open_api()

In [ ]:
import cmr.auth.token as t
import cmr.search.collection as coll

import platform
#print ("Found in " + platform.python_version())
import cmr.util.common as com

This is starting to get complicated with the data behind an ACL. I can't get the bearer token to work here (and it involves going to URS and generating a token on my profile page - how would we time this?)

In [ ]:
params = {}
#config = {'env':'uat'}
#params['provider'] = 'NSIDC_TS1'
params['collection_concept_id'] = 'C1233596099-NSIDC_TS1' 
#params['bounding_box']= '-10,-5,10,5'
#results = gran.search(params, config=config)
results = gran.search(params, config=t.use_bearer_token(config={'env': 'uat'}))
print("Found {} records.".format(len(results)))
for i in results:
    print (i)

Back to the drawing board... trying the old fashioned way with the CMR API, and using https://github.com/podaac/AGU-2020/blob/main/Part-II/04_melt_pond/melt-pond-tutorial.ipynb as a framework.

## __On Prem workflow__

### 1. Authenticate against Earthdata Login


In [2]:
def setup_earthdata_login_auth(endpoint: str='urs.uat.earthdata.nasa.gov'):
    netrc_name = "_netrc" if system()=="Windows" else ".netrc"
    try:
        username, _, password = netrc(file=join(expanduser('~'), netrc_name)).authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        print('Please provide your Earthdata Login credentials for access.')
        print('Your info will only be passed to %s and will not be exposed in Jupyter.' % (endpoint))
        username = input('Username: ')
        password = getpass('Password: ')
    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)
    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)
    
# Start authenticated session with URS to allow restricted data downloads:
setup_earthdata_login_auth(endpoint="urs.uat.earthdata.nasa.gov")

Please provide your Earthdata Login credentials for access.
Your info will only be passed to urs.uat.earthdata.nasa.gov and will not be exposed in Jupyter.


Username:  amy.steiker
Password:  ·········


### 2. Set up CMR token auth

In [3]:
TOKEN_DATA = ("<token>"
              "<username>%s</username>"
              "<password>%s</password>"
              "<client_id>NSIDC TS1 Client</client_id>"
              "<user_ip_address>%s</user_ip_address>"
              "</token>")


def setup_cmr_token_auth(endpoint: str='cmr.uat.earthdata.nasa.gov'):
    ip = requests.get("https://ipinfo.io/ip").text.strip()
    return requests.post(
        url="https://%s/legacy-services/rest/tokens" % endpoint,
        data=TOKEN_DATA % (input("Username: "), getpass("Password: "), ip),
        headers={'Content-Type': 'application/xml', 'Accept': 'application/json'}
    ).json()['token']['id']





# Get your authentication token for searching restricted records in the CMR:
_token = setup_cmr_token_auth(endpoint="cmr.uat.earthdata.nasa.gov")

Username:  amy.steiker
Password:  ·········


### 3. Declare time and bounding box search parameters

_Using a bounding box that will cover granules in EEDTEST (limiting factor compared to data in TS1)_

In [4]:
# Bounding Box spatial parameter in decimal degree 'W,S,E,N' format.
bounding_box = '-13.1,-72.7,114.5,-65.5'

# Each date in yyyy-MM-ddTHH:mm:ssZ format; date range in start,end format
temporal = '2020-01-01T00:00:00Z,2020-02-10T23:59:59Z'

search_parameters= {'short_name': 'ATL03',
               'version': '003', 
               'provider': 'NSIDC_TS1',
               'bounding_box': bounding_box,
               'temporal': temporal,
                   }

### 4. Search for granules based on time and bounding box

In [5]:
def search_granules(search_parameters, token, geojson=None, output_format="json"):
    """
    Performs a granule search with token authentication for restricted results
    
    :search_parameters: dictionary of CMR search parameters
    :token: CMR token needed for restricted search
    :geojson: filepath to GeoJSON file for spatial search
    :output_format: select format for results https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#supported-result-formats
    
    :returns: if hits is greater than 0, search results are returned in chosen output_format, otherwise returns None.
    """
    search_url = "https://cmr.uat.earthdata.nasa.gov/search/granules"
    
    # add token to search parameters
    search_parameters['token'] = token
    
    if geojson:
        files = {"shapefile": (geojson, open(geojson, "r"), "application/geo+json")}
    else:
        files = None
    
    
    parameters = {
        "scroll": "true",
        "page_size": 100,
    }
    
    try:
        response = requests.post(f"{search_url}.{output_format}", params=parameters, data=search_parameters, files=files)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f"HTTP Error: {http_error}")
    except Exception as err:
        print(f"Error: {err}")
    
    hits = int(response.headers['CMR-Hits'])
    if hits > 0:
        print(f"Found {hits} files")
        results = json.loads(response.content)
        granules = []
        granules.extend(results['feed']['entry'])
        granule_sizes = [float(granule['granule_size']) for granule in granules]
        print(f"The total size of all files is {sum(granule_sizes):.2f} MB")
        return response.json()
    else:
        print("Found no hits")
        return

search_granules(search_parameters, _token)

Found 2 files
The total size of all files is 1991.09 MB


{'feed': {'updated': '2021-07-16T20:44:33.711Z',
  'id': 'https://cmr.uat.earthdata.nasa.gov:443/search/granules.json?short_name=ATL03&version=003&provider=NSIDC_TS1&bounding_box=-13.1%2C-72.7%2C114.5%2C-65.5&temporal=2020-01-01T00%3A00%3A00Z%2C2020-02-10T23%3A59%3A59Z&token=B1782358-CEBF-F218-CB49-B6FFD3DA2DE1',
  'title': 'ECHO granule metadata',
  'entry': [{'producer_granule_id': 'ATL03_20200131230704_05530610_003_01.h5',
    'time_start': '2020-01-31T23:07:03.945Z',
    'orbit': {'ascending_crossing': '-77.9931326446199',
     'start_lat': '-50',
     'start_direction': 'D',
     'end_lat': '-79',
     'end_direction': 'D'},
    'updated': '2021-06-15T09:44:18.316Z',
    'orbit_calculated_spatial_domains': [{'equator_crossing_date_time': '2020-01-31T22:06:55.775Z',
      'equator_crossing_longitude': '-77.9931326446199',
      'orbit_number': '7689'}],
    'dataset_id': 'ATLAS/ICESat-2 L2A Global Geolocated Photon Data V003',
    'data_center': 'NSIDC_TS1',
    'title': 'SC:ATL03.

### 5. Determine subsetting service options

In [6]:
def search_services(search_parameters, token, output_format="json"):
    """
    Performs a granule search with token authentication for restricted results
    
    :search_parameters: dictionary of CMR search parameters
    :token: CMR token needed for restricted search
    :output_format: select format for results https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#supported-result-formats
    
    :returns: if hits is greater than 0, search results are returned in chosen output_format, otherwise returns None.
    """
    collection_url = "https://cmr.uat.earthdata.nasa.gov/search/collections"
    
    # add token to search parameters
    search_parameters['token'] = token
    
    parameters = {
        "scroll": "true",
        "page_size": 100,
    }
    
    try:
        response = requests.post(f"{collection_url}.{output_format}", params=parameters, data=search_parameters)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f"HTTP Error: {http_error}")
    except Exception as err:
        print(f"Error: {err}")
    
    hits = int(response.headers['CMR-Hits'])
    if hits > 0:
        response = response.json()
        if 'services' in response['feed']['entry'][0]['associations']: 
            services = response['feed']['entry'][0]['associations']['services']
            output_format = "umm_json"
            service_url = "https://cmr.uat.earthdata.nasa.gov/search/services"
            for i in range(len(services)):
                response = requests.get(f"{service_url}.{output_format}?concept-id={services[i]}")
                response = response.json()
                if 'ServiceOptions' in response['items'][0]['umm']: pprint(response['items'][0]['umm']['ServiceOptions'])
        else:
            print("Found no services")
        return
    else:
        print("Found no hits")
        return

search_services(search_parameters, _token)

{'InterpolationTypes': ['Bicubic Interpolation'],
 'Subset': {'TemporalSubset': {'AllowMultipleValues': False}},
 'SupportedReformattings': [{'SupportedInputFormat': 'HDF5',
                             'SupportedOutputFormats': ['ASCII',
                                                        'NETCDF-3',
                                                        'NETCDF-4',
                                                        'Shapefile']}],
 'VariableAggregationSupportedMethods': ['COUNT']}


### 6. Determine variables

_Today there are no UMM-Var records for the on-prem collection_

#### 7. Determine variables: Authenticate against EGI endpoint

In [7]:
uid = input('Earthdata Login user name: ') # Enter Earthdata Login user name
pswd = getpass('Earthdata Login password: ') # Enter Earthdata Login password

# Query service capability URL 

from xml.etree import ElementTree as ET

capability_url = f'https://n5eil12u.ecs.nsidc.org/egi_TS1/capabilities/{search_parameters["short_name"]}.{search_parameters["version"]}.xml'

# Create session to store cookie and pass credentials to capabilities url

session = requests.session()
s = session.get(capability_url)
response = session.get(s.url,auth=(uid,pswd))


Earthdata Login user name:  amy.steiker
Earthdata Login password:  ·········


#### 8. Determine variables: Query EGI for variables

In [8]:
root = ET.fromstring(response.content)

#collect lists with each service option

subagent = [subset_agent.attrib for subset_agent in root.iter('SubsetAgent')]
if len(subagent) > 0 :

    # variable subsetting
    variables = [SubsetVariable.attrib for SubsetVariable in root.iter('SubsetVariable')]  
    variables_raw = [variables[i]['value'] for i in range(len(variables))]
    variables_join = [''.join(('/',v)) if v.startswith('/') == False else v for v in variables_raw] 
    variable_vals = [v.replace(':', '/') for v in variables_join]

    # reformatting
    formats = [Format.attrib for Format in root.iter('Format')]
    format_vals = [formats[i]['value'] for i in range(len(formats))]
    format_vals.remove('')

    # reprojection options
    projections = [Projection.attrib for Projection in root.iter('Projection')]
    
print(variable_vals)

['/', '//ds_surf_type', '//ds_xyz', '/ancillary_data', '/ancillary_data/atlas_sdp_gps_epoch', '/ancillary_data/control', '/ancillary_data/data_end_utc', '/ancillary_data/data_start_utc', '/ancillary_data/end_cycle', '/ancillary_data/end_delta_time', '/ancillary_data/end_geoseg', '/ancillary_data/end_gpssow', '/ancillary_data/end_gpsweek', '/ancillary_data/end_orbit', '/ancillary_data/end_region', '/ancillary_data/end_rgt', '/ancillary_data/granule_end_utc', '/ancillary_data/granule_start_utc', '/ancillary_data/release', '/ancillary_data/start_cycle', '/ancillary_data/start_delta_time', '/ancillary_data/start_geoseg', '/ancillary_data/start_gpssow', '/ancillary_data/start_gpsweek', '/ancillary_data/start_orbit', '/ancillary_data/start_region', '/ancillary_data/start_rgt', '/ancillary_data/version', '/ancillary_data/altimetry', '/ancillary_data/altimetry/atl03_pad', '/ancillary_data/altimetry/band_tol', '/ancillary_data/altimetry/min_full_sat', '/ancillary_data/altimetry/min_near_sat', '

### 9. Request variable subset

_Starting with a variable subsetting request, since spatial subsetting for ICESat-2 is not available yet in the cloud. Alternatively we could attempt to spatially subset without services (i.e. subsetting using xarray)_

#### Request variable subset: Determine API request 

_This isn't really required so not including it as a step_

In [9]:
#Set NSIDC data access base URL
base_url = 'https://n5eil12u.ecs.nsidc.org/egi_TS1/request'

# bounding box search and subset:
param_dict = {'short_name': 'ATL03', 
              'version': '003', 
              'temporal': temporal, 
              'bounding_box': bounding_box, 
              'bbox': bounding_box, 
              'coverage': '/gt1r/heights/h_ph,/gt1l/heights/h_ph,/gt2r/heights/h_ph,/gt2l/heights/h_ph,/gt3r/heights/h_ph,/gt3l/heights/h_ph',
              'page_size': '10', 
              'request_mode': 'async',
              'email': '',
              'token' : _token,
             }

#Remove blank key-value-pairs
param_dict = {k: v for k, v in param_dict.items() if v != ''}

#Convert to string
param_string = '&'.join("{!s}={!r}".format(k,v) for (k,v) in param_dict.items())
param_string = param_string.replace("'","")

API_request = api_request = f'{base_url}?{param_string}'
print(API_request)

https://n5eil12u.ecs.nsidc.org/egi_TS1/request?short_name=ATL03&version=003&temporal=2020-01-01T00:00:00Z,2020-02-10T23:59:59Z&bounding_box=-13.1,-72.7,114.5,-65.5&bbox=-13.1,-72.7,114.5,-65.5&coverage=/gt1r/heights/h_ph,/gt1l/heights/h_ph,/gt2r/heights/h_ph,/gt2l/heights/h_ph,/gt3r/heights/h_ph,/gt3l/heights/h_ph&page_size=10&request_mode=async&token=B1782358-CEBF-F218-CB49-B6FFD3DA2DE1


#### 10. Request variable subset: Download customized data

In [10]:

# Create an output folder if the folder does not already exist.

path = str(os.getcwd() + '/Outputs')
if not os.path.exists(path):
    os.mkdir(path)

# For all requests other than spatial file upload, use get function
request = session.get(base_url, params=param_dict)

print('Request HTTP response: ', request.status_code)

# Raise bad request: Loop will stop for bad response code.
request.raise_for_status()
print('Order request URL: ', request.url)
esir_root = ET.fromstring(request.content)
print('Order request response XML content: ', request.content)

#Look up order ID
orderlist = []   
for order in esir_root.findall("./order/"):
    orderlist.append(order.text)
orderID = orderlist[0]
print('order ID: ', orderID)

#Create status URL
statusURL = base_url + '/' + orderID
print('status URL: ', statusURL)

#Find order status
request_response = session.get(statusURL)    
print('HTTP response from order response URL: ', request_response.status_code)

# Raise bad request: Loop will stop for bad response code.
request_response.raise_for_status()
request_root = ET.fromstring(request_response.content)
statuslist = []
for status in request_root.findall("./requestStatus/"):
    statuslist.append(status.text)
status = statuslist[0]
print('Data request is submitting...')
print('Initial request status is ', status)

#Continue loop while request is still processing
while status == 'pending' or status == 'processing': 
    print('Status is not complete. Trying again.')
    time.sleep(10)
    loop_response = session.get(statusURL)

# Raise bad request: Loop will stop for bad response code.
    loop_response.raise_for_status()
    loop_root = ET.fromstring(loop_response.content)

#find status
    statuslist = []
    for status in loop_root.findall("./requestStatus/"):
        statuslist.append(status.text)
    status = statuslist[0]
    print('Retry request status is: ', status)
    if status == 'pending' or status == 'processing':
        continue

#Order can either complete, complete_with_errors, or fail:
# Provide complete_with_errors error message:
if status == 'complete_with_errors' or status == 'failed':
    messagelist = []
    for message in loop_root.findall("./processInfo/"):
        messagelist.append(message.text)
    print('error messages:')
    pprint.pprint(messagelist)

# Download zipped order if status is complete or complete_with_errors
if status == 'complete' or status == 'complete_with_errors':
    downloadURL = 'https://n5eil12u.ecs.nsidc.org/esir_TS1/' + orderID + '.zip'
    print('Zip download URL: ', downloadURL)
    print('Beginning download of zipped output...')
    zip_response = session.get(downloadURL)
    # Raise bad request: Loop will stop for bad response code.
    zip_response.raise_for_status()
    with zipfile.ZipFile(io.BytesIO(zip_response.content)) as z:
        z.extractall(path)
    print('Data request is complete.')
else: print('Request failed.')

Request HTTP response:  201
Order request URL:  https://n5eil12u.ecs.nsidc.org/egi_TS1/request?short_name=ATL03&version=003&temporal=2020-01-01T00%3A00%3A00Z%2C2020-02-10T23%3A59%3A59Z&bounding_box=-13.1%2C-72.7%2C114.5%2C-65.5&bbox=-13.1%2C-72.7%2C114.5%2C-65.5&coverage=%2Fgt1r%2Fheights%2Fh_ph%2C%2Fgt1l%2Fheights%2Fh_ph%2C%2Fgt2r%2Fheights%2Fh_ph%2C%2Fgt2l%2Fheights%2Fh_ph%2C%2Fgt3r%2Fheights%2Fh_ph%2C%2Fgt3l%2Fheights%2Fh_ph&page_size=10&request_mode=async&token=B1782358-CEBF-F218-CB49-B6FFD3DA2DE1
Order request response XML content:  b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<eesi:agentResponse xmlns="" xmlns:iesi="http://eosdis.nasa.gov/esi/rsp/i" xmlns:ssw="http://newsroom.gsfc.nasa.gov/esi/rsp/ssw" xmlns:eesi="http://eosdis.nasa.gov/esi/rsp/e" xmlns:esi="http://eosdis.nasa.gov/esi/rsp" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n    <order>\n        <orderId>21954</orderId>\n        <Instructions>You may receive an email about your order if you speci

#### 11. Clean up Outputs folder by removing individual granule folders 

In [ ]:
for root, dirs, files in os.walk(path, topdown=False):
    for file in files:
        try:
            shutil.move(os.path.join(root, file), path)
        except OSError:
            pass
    for name in dirs:
        os.rmdir(os.path.join(root, name))

### 12. Read in Data using h5py

In [11]:
def load_icesat2_as_dataframe(filepath, VARIABLES):
    '''
    Load points from an ICESat-2 granule 'gt<beam>' groups as DataFrame of points. Uses VARIABLES mapping
    to select subset of '/gt<beam>/...' variables  (Assumes these variables share dimensions)
    Arguments:
        filepath to ATL0# granule
    '''
    
    ds = h5py.File(filepath, 'r')

    # Get dataproduct name
    dataproduct = ds.attrs['identifier_product_type'].decode()
    # Convert variable paths to 'Path' objects for easy manipulation
    variables = [Path(v) for v in VARIABLES[dataproduct]]
    # Get set of beams to extract individially as dataframes combining in the end
    beams = {list(v.parents)[-2].name for v in variables}
    
    dfs = []
    for beam in beams:
        data_dict = {}
        beam_variables = [v for v in variables if beam in str(v)]
        for variable in beam_variables:
            # Use variable 'name' as column name. Beam will be specified in 'beam' column
            column = variable.name
            variable = str(variable)
            try:
                values = ds[variable][:]
                # Convert invalid data to np.nan (only for float columns)
                if 'float' in str(values.dtype):
                    if 'valid_min' in ds[variable].attrs:
                        values[values < ds[variable].attrs['valid_min']] = np.nan
                    if 'valid_max' in ds[variable].attrs:
                        values[values > ds[variable].attrs['valid_max']] = np.nan
                    if '_FillValue' in ds[variable].attrs:
                        values[values == ds[variable].attrs['_FillValue']] = np.nan
                    
                data_dict[column] = values
            except KeyError:
                print(f'Variable {variable} not found in {filepath}. Likely an empty granule.')
                raise
                
        df = pd.DataFrame.from_dict(data_dict)
        df['beam'] = beam
        dfs.append(df)
        
    df = pd.concat(dfs, sort=True)
    # Add filename column for book-keeping and reset index
    df['filename'] = Path(filepath).name
    df = df.reset_index(drop=True)
    
    return df

In [20]:
atl_filepath = './Outputs/processed_ATL03_20200131230704_05530610_003_01.h5' # Define local filepath 

VARIABLES = {
    'ATL03': [
        '/gt1l/heights/h_ph',
        '/gt1r/heights/h_ph',
        '/gt2l/heights/h_ph',
        '/gt2r/heights/h_ph',
        '/gt3l/heights/h_ph',
        '/gt3r/heights/h_ph',
        ]
}

atl03 = load_icesat2_as_dataframe(atl_filepath, VARIABLES)
atl03.tail()

,beam,h_ph,filename
15855052,gt1l,2601.556641,processed_ATL03_20200131230704_05530610_003_01.h5
15855053,gt1l,2571.268066,processed_ATL03_20200131230704_05530610_003_01.h5
15855054,gt1l,2601.609131,processed_ATL03_20200131230704_05530610_003_01.h5
15855055,gt1l,2644.662109,processed_ATL03_20200131230704_05530610_003_01.h5
15855056,gt1l,2594.167236,processed_ATL03_20200131230704_05530610_003_01.h5


### 13. Plot data

In [28]:
# # #Convert sst data array to masked array:
# # lon = ma.masked_invalid(sst['lon'])
# # lat = ma.masked_invalid(sst['lat'])
# # sst_array = ma.masked_invalid(sst['sea_surface_temperature'])

# map_proj = cartopy.crs.NorthPolarStereo()
# data_proj = cartopy.crs.PlateCarree()

# fig = plt.figure(figsize=(14,6))
# ax = fig.add_subplot(111, projection=map_proj)
# ax.coastlines(color='red')
# # sst_plot = ax.pcolor(lon,\
# #                       lat,\
# #                       sst_array[0],\
# #                       transform=data_proj)
# # #ax.set_extent([-62.5,-53.5,81.5,83.5])

# # add colorbar
# axpos = ax.get_position()
# # cbar_ax = fig.add_axes([axpos.x1+0,axpos.y0,0.03,axpos.height])
# # cbar = fig.colorbar(sst_plot, cax=cbar_ax)
# # cbar.ax.tick_params(labelsize=12)
# # cbar.set_label('sst (kelvin)', fontsize=12)

# height_plot = ax.scatter(atl03.longitude,\
#                         atl03.latitude,\
#                         c=atl03.height_segment_height,\
#                         vmax=1.5,\
#                         cmap='Reds',\
#                         alpha=0.7,\
#                         s=1,\
#                         transform=data_proj)

## __Cloud workflow__


### 1. Set up CMR token auth

In [29]:
TOKEN_DATA = ("<token>"
              "<username>%s</username>"
              "<password>%s</password>"
              "<client_id>NSIDC TS1 Client</client_id>"
              "<user_ip_address>%s</user_ip_address>"
              "</token>")


def setup_cmr_token_auth(endpoint: str='cmr.uat.earthdata.nasa.gov'):
    ip = requests.get("https://ipinfo.io/ip").text.strip()
    return requests.post(
        url="https://%s/legacy-services/rest/tokens" % endpoint,
        data=TOKEN_DATA % (input("Username: "), getpass("Password: "), ip),
        headers={'Content-Type': 'application/xml', 'Accept': 'application/json'}
    ).json()['token']['id']


# Get your authentication token for searching restricted records in the CMR:
_token = setup_cmr_token_auth(endpoint="cmr.uat.earthdata.nasa.gov")

Username:  amy.steiker
Password:  ·········


### 2. Declare time and bounding box search parameters

_Using a bounding box that will cover granules in EEDTEST (limiting factor compared to data in TS1)_

In [30]:
# Bounding Box spatial parameter in decimal degree 'W,S,E,N' format.
bounding_box = '-13.1,-72.7,114.5,-65.5'

# Each date in yyyy-MM-ddTHH:mm:ssZ format; date range in start,end format
temporal = '2020-01-01T00:00:00Z,2020-02-10T23:59:59Z'

In [31]:
search_parameters= {'short_name': 'ATL03VARSUBSETTER',
               'version': '003', 
               #'concept_id':'C1234714691-EEDTEST',
               'provider': 'EEDTEST',
               'bounding_box': bounding_box,
               'temporal': temporal,
                   }

### 3. Search for granules based on time and bounding box

In [32]:
def search_granules(search_parameters, token, geojson=None, output_format="json"):
    """
    Performs a granule search with token authentication for restricted results
    
    :search_parameters: dictionary of CMR search parameters
    :token: CMR token needed for restricted search
    :geojson: filepath to GeoJSON file for spatial search
    :output_format: select format for results https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#supported-result-formats
    
    :returns: if hits is greater than 0, search results are returned in chosen output_format, otherwise returns None.
    """
    search_url = "https://cmr.uat.earthdata.nasa.gov/search/granules"
    
    # add token to search parameters
    search_parameters['token'] = token
    
    if geojson:
        files = {"shapefile": (geojson, open(geojson, "r"), "application/geo+json")}
    else:
        files = None
    
    
    parameters = {
        "scroll": "true",
        "page_size": 100,
    }
    
    try:
        response = requests.post(f"{search_url}.{output_format}", params=parameters, data=search_parameters, files=files)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f"HTTP Error: {http_error}")
    except Exception as err:
        print(f"Error: {err}")
    
    hits = int(response.headers['CMR-Hits'])
    if hits > 0:
        print(f"Found {hits} files")
        results = json.loads(response.content)
        granules = []
        granules.extend(results['feed']['entry'])
        granule_sizes = [float(granule['granule_size']) for granule in granules]
        print(f"The total size of all files is {sum(granule_sizes):.2f} MB")
        return response.json()
    else:
        print("Found no hits")
        return

search_granules(search_parameters, _token)

Found 2 files
The total size of all files is 1991.09 MB


{'feed': {'updated': '2021-07-16T21:04:54.053Z',
  'id': 'https://cmr.uat.earthdata.nasa.gov:443/search/granules.json?short_name=ATL03VARSUBSETTER&version=003&provider=EEDTEST&bounding_box=-13.1%2C-72.7%2C114.5%2C-65.5&temporal=2020-01-01T00%3A00%3A00Z%2C2020-02-10T23%3A59%3A59Z&token=AC9CF2CC-E144-80BC-3AA9-53C9E92E8075',
  'title': 'ECHO granule metadata',
  'entry': [{'producer_granule_id': 'ATL03_20200131230704_05530610_003_01.h5',
    'time_start': '2020-01-31T23:07:03.945Z',
    'orbit': {'ascending_crossing': '-77.9931326446199',
     'start_lat': '-50',
     'start_direction': 'D',
     'end_lat': '-79',
     'end_direction': 'D'},
    'updated': '2020-05-07T12:12:21.648Z',
    'orbit_calculated_spatial_domains': [{'equator_crossing_date_time': '2020-01-31T22:06:55.775Z',
      'equator_crossing_longitude': '-77.9931326446199',
      'orbit_number': '7689'}],
    'dataset_id': 'ATLAS-ICESat-2 L2A Global Geolocated Photon Data V003',
    'data_center': 'EEDTEST',
    'title': 'E

### 4. Determine subsetting service options

In [33]:
def search_services(search_parameters, token, output_format="json"):
    """
    Performs a granule search with token authentication for restricted results
    
    :search_parameters: dictionary of CMR search parameters
    :token: CMR token needed for restricted search
    :output_format: select format for results https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#supported-result-formats
    
    :returns: if hits is greater than 0, search results are returned in chosen output_format, otherwise returns None.
    """
    collection_url = "https://cmr.uat.earthdata.nasa.gov/search/collections"
    
    # add token to search parameters
    search_parameters['token'] = token
    
    parameters = {
        "scroll": "true",
        "page_size": 100,
    }
    
    try:
        response = requests.post(f"{collection_url}.{output_format}", params=parameters, data=search_parameters)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f"HTTP Error: {http_error}")
    except Exception as err:
        print(f"Error: {err}")
    
    hits = int(response.headers['CMR-Hits'])
    if hits > 0:
        response = response.json()
        if 'services' in response['feed']['entry'][0]['associations']: 
            services = response['feed']['entry'][0]['associations']['services']
            output_format = "umm_json"
            service_url = "https://cmr.uat.earthdata.nasa.gov/search/services"
            for i in range(len(services)):
                response = requests.get(f"{service_url}.{output_format}?concept-id={services[i]}")
                response = response.json()
                if 'ServiceOptions' in response['items'][0]['umm']: pprint(response['items'][0]['umm']['ServiceOptions'])
        else:
            print("Found no services")
        return
    else:
        print("Found no hits")
        return

search_services(search_parameters, _token)

{'Subset': {'SpatialSubset': {'BoundingBox': {'AllowMultipleValues': False}},
            'TemporalSubset': {'AllowMultipleValues': False},
            'VariableSubset': {'AllowMultipleValues': True}},
 'SupportedInputProjections': [{'ProjectionName': 'Geographic'}],
 'SupportedOutputProjections': [{'ProjectionName': 'Geographic'}]}
{'Subset': {'SpatialSubset': {'BoundingBox': {'AllowMultipleValues': False}},
            'TemporalSubset': {'AllowMultipleValues': False},
            'VariableSubset': {'AllowMultipleValues': True}},
 'SupportedInputProjections': [{'ProjectionName': 'Geographic'}],
 'SupportedOutputProjections': [{'ProjectionName': 'Geographic'}],
 'SupportedReformattings': [{'SupportedInputFormat': 'NETCDF-4',
                             'SupportedOutputFormats': ['ASCII',
                                                        'BINARY',
                                                        'GEOTIFF',
                                                        'NETCDF-3',

### 5. Determine variables

_Can use UMM-Var to return cleaner Long Names, and Name value needed for Harmony request_

In [38]:
def search_vars(search_parameters, token, output_format="json"):
    """
    Performs a granule search with token authentication for restricted results
    
    :search_parameters: dictionary of CMR search parameters
    :token: CMR token needed for restricted search
    :output_format: select format for results https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#supported-result-formats
    
    :returns: if hits is greater than 0, search results are returned in chosen output_format, otherwise returns None.
    """
    collection_url = "https://cmr.uat.earthdata.nasa.gov/search/collections"
    
    # add token to search parameters
    search_parameters['token'] = token
    
    parameters = {
        "scroll": "true",
        "page_size": 100,
    }
    
    try:
        response = requests.post(f"{collection_url}.{output_format}", params=parameters, data=search_parameters)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f"HTTP Error: {http_error}")
    except Exception as err:
        print(f"Error: {err}")
    
    hits = int(response.headers['CMR-Hits'])
    if hits > 0:
        response = response.json()
        if 'variables' in response['feed']['entry'][0]['associations']: 
            variables = response['feed']['entry'][0]['associations']['variables']
            output_format = "umm_json"
            var_url = "https://cmr.uat.earthdata.nasa.gov/search/variables"
            for i in range(len(variables)):
                response = requests.get(f"{var_url}.{output_format}?concept-id={variables[i]}")
                response = response.json()
                if 'LongName' in response['items'][0]['umm']: 
                    pprint(response['items'][0]['umm']['LongName'])
                    pprint(response['items'][0]['umm']['Name'])
        else:
            print("Found no services")
        return
    else:
        print("Found no hits")
        return

search_vars(search_parameters, _token)

'background counts per bin'
'/gt1r/signal_find_output/ocean/bckgrd_mean'
'Segment latitude'
'/gt1r/geolocation/reference_photon_lat'
'Geoid'
'/gt1r/geophys_corr/geoid'
'Latitude'
'/gt1r/heights/lat_ph'
'Photon WGS84 Height'
'/gt1r/heights/h_ph'
'Longitude'
'/gt1r/heights/lon_ph'
'ATLAS 50-shot background count'
'/gt1r/bckgrd_atlas/bckgrd_counts'
'Segment longitude'
'/gt1r/geolocation/reference_photon_lon'
'Orbit Number'
'/orbit_info/orbit_number'
'Altitude'
'/gt1r/geolocation/altitude_sc'
'Photon Index Begin'
'/gt1r/geolocation/ph_index_beg'


### 6. Request variable subset

_Starting with a variable subsetting request, since spatial subsetting for ICESat-2 is not available yet in the cloud. Alternatively we could attempt to spatially subset without services (i.e. subsetting using xarray)_

_Using Harmony-py, this is a few easy lines of code_

In [35]:
!{sys.executable} -m pip install -U harmony-py #install harmony-py into Python kernel
import sys; sys.path.append('..')
import datetime as dt
from IPython.display import display, JSON
import rasterio
import rasterio.plot

from harmony import BBox, Client, Collection, Request
from harmony.config import Environment

  Attempting uninstall: harmony-py
    Found existing installation: harmony-py 0.1.2
    Uninstalling harmony-py-0.1.2:
      Successfully uninstalled harmony-py-0.1.2


In [45]:
harmony_client = Client(auth=('uid', 'pwd'), env=Environment.UAT)
request = Request(
    collection=Collection(id='ATL03VARSUBSETTER'),
    spatial=BBox(-13.1,-72.7,114.5,-65.5),
    temporal={
        'start': dt.datetime(2020, 1, 1),
        'stop': dt.datetime(2020, 2, 10)
    },
    variables=['/gt1r/heights/h_ph','/gt1r/geolocation/reference_photon_lat','/gt1r/geophys_corr/geoid','/gt1r/heights/lon_ph','/gt1r/geolocation/reference_photon_lon','/gt1r/geolocation/ph_index_beg'],
)
job_id = harmony_client.submit(request)
job_id
#results = harmony_client.download_all(job_id, directory='/tmp', overwrite=True)

### 7. Retrieve AWS Credentials

In [48]:
creds = harmony_client.aws_credentials()
creds

### 8. Plot data in place

In [ ]:
stac_catalog_url = harmony_client.stac_catalog_url(job_id)

import intake
cat = intake.open_stac_catalog(stac_catalog_url)
display(list(cat))

#
# NOTE: Execution of this cell will only succeed within the AWS us-west-2 region. 
#

for i in range(len(list(cat))):
    da = cat[list(cat)[i]][entries[i].describe()['name']].to_dask()
    display(da)
    da.plot()